In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(), # PIL Image를 Tensor로 변환하는 Transform. 이미지의 픽셀값을 [0, 255]에서 [0.0, 1.0] 범위로 스케일링한다.
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # transforms.Compose 뒤에는 list of "Transform objects"가 와야 함.
                                                              # transform은 다양한 Data augmetation을 한번에 손쉽게 해주는 기능이다. 

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform) # 얘는 사실상 trainset 변수에 데이터를 저장한 것이 아니라 특정 위치에 데이터가 있고 그걸 불러오는 역할을 하는 객체를 trainset 변수에 저장한 것
                                                                             # testset[0]과 같이 입력하면 특정 데이터를 불러오는 것임.
                                                                             # 다운로드가 필요하다면 download=True로 설정
                                                                             # Train set으로 데이터를 다운 받음
                                                                              
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)      # batch size만큼 데이터를 묶어서 로드, shuffle=True로 데이터 섞음, num_workers=2로 subprocess 사용하여 데이터 로드 
                                                                            # 아예 별도의 메모리 공간을 갖는 '프로세스'를 새로 띄워서(Subprocess) 물리적으로 CPU의 여러 코어를 동시에 사용하는 방식을 택합니다.

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 입력 채널 수 3(RGB), 출력 채널 수 6(feature map이 6개), 커널 크기 5(5*5 필터)인 2D convolutional layer
        self.pool = nn.MaxPool2d(2, 2) # 커널 크기 2, 스트라이드 2인 Max pooling layer. 입력을 2x2 영역으로 나누고 각 영역에서 최대값을 출력하여 공간 크기를 절반으로 줄임
        self.conv2 = nn.Conv2d(6, 16, 5) # conv1에서 출력된 6개의 feature map을 입력으로 받아서 16개의 feature map을 출력하는 2D convolutional layer 
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # flatten된 feature map을 입력으로 받아서 120개의 출력을 내놓는 fully connected layer. conv2의 출력이 16개의 feature map이고, 각 feature map의 크기가 5x5이므로, 총 16*5*5개의 입력이 fc1에 들어감
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 가장 높은 점수의 인덱스를 고르면 됨.
        return x


net = Net()

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# 여기서 0은 첫 번째 GPU를 의미합니다. 만약 여러 개의 GPU가 있다면, 'cuda:1', 'cuda:2' 등으로 지정할 수 있습니다.
# CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력합니다:

print(device)

# 사용 가능한 GPU 개수 출력
gpu_count = torch.cuda.device_count()
print(f"사용 가능한 GPU 개수: {gpu_count}")

cuda:0
사용 가능한 GPU 개수: 1


In [12]:
# 모든 모듈의 매개변수와 버퍼를 CUDA 장치로 이동
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # Softmax가 내부에서 수행.
                                  # CrossEntropyLoss는 모델의 출력과 실제 레이블 간의 차이를 계산하는 손실 함수
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 옵티마이저는 gpu에 올릴 필요 X
                                                                # 이미 모델이 gpu에 올라가 있으면 옵티마이저도 gpu 주소 자동 참조


In [15]:
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # enumerate(trainloader, 0)은 trainloader에서 데이터를 하나씩 꺼내면서 인덱스도 함께 반환하는 함수입니다. i는 인덱스, data는 실제 데이터입니다.
                                              # 그냥 trainloader를 호출해도 for문이 돌아감. for문이 알아서 iter해줌.
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;

        inputs, labels = data[0].to(device), data[1].to(device)
        
        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels) # 위에서 정의한 cross entropy loss 함수 사용
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.198
[1,  4000] loss: 1.192
[1,  6000] loss: 1.196
[1,  8000] loss: 1.206
[1, 10000] loss: 1.209
[1, 12000] loss: 1.175
[2,  2000] loss: 1.082
[2,  4000] loss: 1.118
[2,  6000] loss: 1.101
[2,  8000] loss: 1.114
[2, 10000] loss: 1.084
[2, 12000] loss: 1.096
Finished Training
